In [3]:
from OthelloEnv import OthelloEnv

import gym
import torch as th
import torch.nn as nn

from stable_baselines3 import A2C
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.env_checker import check_env

In [4]:
env = OthelloEnv()
check_env(env)

In [ ]:
class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 256):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 256, kernel_size=(8, 8), stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=(4, 4), stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=(2, 2), stride=1, padding='same'),
            nn.ReLU(),
            nn.Flatten()
        )

        with th.no_grad():
            obs = th.as_tensor(observation_space.sample()[None])
            n_flatten = self.cnn(obs.float()).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=256),
)

model = A2C('CnnPolicy', env, policy_kwargs=policy_kwargs, verbose=0, tensorboard_log='logs')
model.learn(100000)

/Users/jacobswe/Python-Projects/Othello-AI/venv/lib/python3.9/site-packages/torch/nn/modules/conv.py:442: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  return F.conv2d(input, weight, bias, self.stride,
